In [1]:
import cv2
import numpy as np
import pandas as pd

import pytesseract
from pytesseract import Output

import re

import skimage.filters as filters
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
import PIL.Image

import os, random, pathlib, warnings, itertools, math
warnings.filterwarnings("ignore")

In [2]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
Testing_Data = r'C:\Users\LENOVO\Desktop\OCR_extraction\Image_extraction\Testing_Data'
Testing_Data = pathlib.Path(Testing_Data)
Front = list(Testing_Data.glob('Front/*'))
Back = list(Testing_Data.glob('Back/*'))

In [ ]:
Front[:5]

In [ ]:
str(Front[:5])

In [ ]:
classes = []
for folder in os.listdir(Testing_Data):
    classes.append(folder)
classes

In [ ]:
size = 500, 300
im = PIL.Image.open(str(Front[1]))
im.resize(size, Image.ANTIALIAS)

In [ ]:
def plot(data_directory,number_of_images,image_folder):
    j=1
    for i in range(number_of_images):
    
        folder = os.path.join(data_directory,image_folder)
        a=random.choice(os.listdir(folder))
        image=PIL.Image.open(os.path.join(folder,a))
        plt.figure(figsize=(10,10))
        plt.subplot(number_of_images,2,j)
        plt.title(label=image_folder, size=17, pad='7.0', loc="center", fontstyle='italic')
        plt.imshow(image)
        j+=1

for j in classes :       
    plot(Testing_Data,1,j)

In [14]:
orig = cv2.imread(r'C:\Users\LENOVO\Desktop\OCR_extraction\Image_extraction\Testing_Data\Front\Ec5QBA4WsAAOq7q.jpg')

In [15]:
mask = np.zeros(orig.shape, dtype=np.uint8)
gray = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
smooth = cv2.GaussianBlur(gray, (95,95), 0)
rgb = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)
rgb1 = Image.frombytes('RGB', orig.shape[:2], orig, 'raw', 'BGR', 0, 0)
division = cv2.divide(gray, smooth, scale=255)
sharp = filters.unsharp_mask(division, radius=1.5, amount=1.5, channel_axis=False, preserve_range=False)
sharp = (255*sharp).clip(0,255).astype(np.uint8)
thresh = cv2.threshold(sharp, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

In [988]:
#cv2.imshow('thresh', thresh)
#cv2.imshow('mask', mask)

In [989]:
# Filter for ROI using contour area and aspect ratio
cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    area = cv2.contourArea(c)
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.05 * peri, True)
    x,y,w,h = cv2.boundingRect(approx)
    aspect_ratio = w / float(h)
    if area > 2000 and aspect_ratio > .5:
        mask[y:y+h, x:x+w] = orig[y:y+h, x:x+w]

In [990]:
tessdata_dir_config = r'C:\Program Files\Tesseract-OCR\tesseract.exe\tessdata' 
custom_config0 = r'--oem 3 --psm 6'
custom_config1 = r'--oem 3 --psm 11'
custom_config2 = r'--oem 3 --psm 8'

In [991]:
# Perfrom OCR with Pytesseract
text1 = pytesseract.image_to_data(thresh,output_type='data.frame',lang='fra', config=custom_config1)
text2 = pytesseract.image_to_string(thresh, lang='fra', config=custom_config1)  #mask or thresh 

In [ ]:
print(text2)

In [993]:
#nums = re.findall(r'^[A-Z]{3,}[0-9]{6,}.', text2)
nums = re.findall(r'[A-Z][0-9][0-9][0-9][0-9][0-9][0-9].|[A-Z][0-9]', text2)
dates = re.findall(r'^(0[1-9]|[12][0-9]|3[01])[- /.](0[1-9]|1[012])[- /.](19|20)\d\d$', text2) #|[1-2][0-9][0-9][0-9]

NAME1 = r"[A-Z][a-z]+,?\s+"
MIDDLE_I = r"(?:[A-Z][a-z]*\.?\s*)?"
NAME2 = r"[A-Z][a-z]+"
names = re.findall(NAME1 + MIDDLE_I + NAME2, text2)

In [ ]:
print("NAME")
for name in names:
	print(name.strip())

In [ ]:
print("ID NUMBERS")
for num in nums:
	print(num.strip())

In [ ]:
print("BIRTH DATE")
for date in dates:
	print(dates.strip())

In [ ]:
plt.imshow(orig)

In [998]:
text = text1[text1.conf != -1]
lines = text.groupby('block_num')['text'].apply(list)
conf = text.groupby(['block_num'])['conf'].mean()

In [999]:
pd.set_option('max_columns', None)
pd.set_option('display.max_rows', text.shape[0]+1)

In [ ]:
print("text1 : \n")
print(text1,"\n\n")

print("text : \n")
print(text, "\n\n")
  
print("lines : \n")
for i in range(len(lines)):
  print("level", i, ": ", lines.iloc[i])

print("\n\n conf : \n")
print(conf)

In [ ]:
d = pytesseract.image_to_data(orig, output_type=Output.DICT, lang='fra')
print(d.keys())

In [1002]:
# Get bounding box estimates
#print(pytesseract.image_to_boxes(Image.open(orig)))

# Get verbose data including boxes, confidences, line and page numbers
#print(pytesseract.image_to_data(Image.open(orig)))

# Get information about orientation and script detection
#print(pytesseract.image_to_osd(Image.open(orig)))

In [1003]:
def display(img,cmap='gray'):
    fig = plt.figure(figsize=(12,10))
    ax = fig.add_subplot(111)
    ax.imshow(img,cmap='gray')

In [ ]:
n_boxes = len(text1['text'])
for i in range(n_boxes):
    if int(text1['conf'][i]) > 45:
        (x, y, w, h) = (text1['left'][i], text1['top'][i], text1['width'][i], text1['height'][i])
        img = cv2.rectangle(orig, (x, y), (x + w, y + h), (0, 255, 0), 2)

display(orig)